**You should not change this cell.**

In [ ]:
import pandas as pd
import numpy as np
import catboost
import pickle
from sklearn.model_selection import KFold
import gc

train = pd.read_csv('/kaggle/input/ioai-2025-preparation-class-lesson-3-homework/train.csv')
train = train.sample(frac = 1.0, random_state = 322).reset_index(drop = True)
test = pd.read_csv('/kaggle/input/ioai-2025-preparation-class-lesson-3-homework/test.csv')
sample_submission = pd.read_csv('/kaggle/input/ioai-2025-preparation-class-lesson-3-homework/sample_submission.csv')

def lgb_train(train, target, split_list, param):

    bst_list = []
    for i , (train_index, test_index) in enumerate(split_list):

        tr = catboost.Pool(train[train_index], label = target[train_index])
        te = catboost.Pool(train[test_index], label = target[test_index])

        bst = catboost.train(tr, param, eval_set = te, iterations = 4000, early_stopping_rounds = 300, verbose =300)
        bst_list += [bst]

        gc.collect()
        del tr, te

    return bst_list

params_cat = {
    'loss_function' :'MultiClass',
    'task_type' : 'GPU',
     'max_depth' : 4,
    'eval_metric' :'MultiClass',
    'learning_rate' : .1,
    'border_count' :  127,
    'random_state' : 42 ,
}

**You can change the function "standart_split" as you want.**

In [ ]:
def standart_split(data, target, n_splits = 5, seed = 322):
    split_list = []
    kf = KFold(n_splits = n_splits, shuffle = True, random_state = seed)
    for train_index, val_index in kf.split(data) :
        split_list += [(train_index, val_index)]
    return split_list

**You can change the set of training columns as you want. To do this, you must add the columns at "drop_cols" list that you do not want to use in the training.**

In [ ]:
drop_cols = ['session_id', 'time', 'target']
train_cols = [x for x in train.columns if x not in drop_cols]
print(len(train_cols))

**If your GPU quota has ended, uncooment this line.**

In [ ]:
# params_cat['task_type'] = 'CPU'

**You should not change this cell. submission.csv generated from that cell you need to use for submit on leaderboard**

In [ ]:
split_list = standart_split(train, 'target')
bst_list = lgb_train(train[train_cols].values, train['target'].values, split_list, params_cat)
pred = []
for bst in bst_list:
    pred += [bst.predict(test[train_cols], prediction_type = 'Probability') ]
sample_submission[sample_submission.columns[1:]] = np.mean(pred, 0)
sample_submission.to_csv('submission.csv', index = None)